# Demonstration for mlflow REST API

This notebook demonstrates use of the mlflow tracking REST api to retrieve results from mlflow experiments and place them into a pandas dataframe.

In [1]:
from __future__ import print_function

In [2]:
import pandas as pd
import numpy as np
import os
import os.path
import requests
import socket

from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.tracking
import mlflow.sklearn

## set up to invoke mlflow tracking REST api

In [4]:
# Assumes MFLOW_TRACKING_URI is set

REST_API_URL = os.environ['MLFLOW_TRACKING_URI'] + '/api/2.0/preview/mlflow'

print(REST_API_URL)

http://db_tracker:5001/api/2.0/preview/mlflow


## List all experiments

In [5]:
r = requests.get(REST_API_URL + '/experiments/list')
experiment_list = r.json()
experiment_list

{'experiments': [{'artifact_location': '/artifacts/0',
   'lifecycle_stage': 'active',
   'name': 'Default',
   'experiment_id': '0'},
  {'artifact_location': '/artifacts/10',
   'lifecycle_stage': 'active',
   'name': 'mlflow_demo1',
   'experiment_id': '10'},
  {'artifact_location': '/artifacts/11',
   'lifecycle_stage': 'active',
   'name': 'mlflow_demo2',
   'experiment_id': '11'}]}

## Retrieve data for specified experiment

In [8]:
r = requests.get(REST_API_URL + '/experiments/get',
                 json={'experiment_id':'11'})
experiment_data = r.json()


In [9]:
experiment_data['experiment']

{'artifact_location': '/artifacts/11',
 'lifecycle_stage': 'active',
 'name': 'mlflow_demo2',
 'experiment_id': '11'}

In [10]:
experiment_metadata = {'experiment_name':experiment_data['experiment']['name'],
                       'experiment_id' : experiment_data['experiment']['experiment_id']}

## Extract experiment results to pandas dataframe

### Retrieve desired experiment attributes

In [11]:
df1 = pd.DataFrame([{k:experiment_data['experiment'][k] for k in ['name','experiment_id']}])
df1

,experiment_id,name
0,11,mlflow_demo2


### Extract run data (metrics, params and tags) 

In [12]:
def extract_run_data(r):
    # seed dataframe row with experiment attributes
    ans = experiment_metadata.copy()
    
    # add run uuid
    ans.update({'run_uuid': r['run_uuid']})
    
    # retrieve metrics, params and tags from the run
    r1 = requests.get(REST_API_URL + '/runs/get',
                      json={'run_uuid': r['run_uuid']})
    run_data = r1.json()['run']['data']
    
    # populate dataframe row with metrics, params and tags 
    for k in run_data.keys():    
        try:
            data = {k+'_'+x['key']:x['value'] for x in run_data[k]}
        except:
            data = {}

        ans.update(data)
        
    # return the dataframe row
    return ans

In [13]:
df = pd.DataFrame([extract_run_data(r) for r in experiment_data['runs']])
df

,experiment_id,experiment_name,metrics_r2,params_algorithm,params_alpha,params_data_set_type,params_hidden_layer_sizes,params_l1_ratio,params_learning_rate,params_max_depth,params_max_iter,params_n_estimators,params_random_state,run_uuid,tags_data_set_type,tags_estimator,tags_mlflow.source.name,tags_mlflow.source.type
0,11,mlflow_demo2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9ab877df8c9c41b1a164a8883f59a495,NaN,NaN,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
1,11,mlflow_demo2,0.229264,ElasticNet,0.009412770080969413,raw values,NaN,0.35833378270496974,NaN,NaN,NaN,NaN,13,645e1e4803d54d1ca0db7c1eab11493b,raw values,from sklearn.linear_model import ElasticNet,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
2,11,mlflow_demo2,0.377038,RandomForestRegressor,NaN,raw values,NaN,NaN,NaN,7,NaN,279,13,ee75aa84c04941769576936c709cebb2,raw values,from sklearn.ensemble import RandomForestRegre...,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
3,11,mlflow_demo2,0.118223,MLPRegressor,NaN,raw values,"(10,)",NaN,NaN,NaN,1422,NaN,13,fa787884e1374b40be9dddebe26450a8,raw values,from sklearn.neural_network import MLPRegressor,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
4,11,mlflow_demo2,0.500365,XGBRegressor,NaN,raw values,NaN,NaN,0.058512491882074746,6,NaN,453,13,a0cf7b8964da457784940f13d09dae79,raw values,from xgboost import XGBRegressor,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
5,11,mlflow_demo2,0.341047,ExtraTreesRegressor,NaN,raw values,NaN,NaN,NaN,7,NaN,279,13,febda8ffd05f4ec8a3ef4e2ae15acf61,raw values,from sklearn.ensemble import ExtraTreesRegressor,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
6,11,mlflow_demo2,0.250190,ElasticNet,0.009412770080969413,standardized values,NaN,0.35833378270496974,NaN,NaN,NaN,NaN,13,f988c23cda0447e7b6347fca111ff77d,standardized values,from sklearn.linear_model import ElasticNet,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
7,11,mlflow_demo2,0.377560,RandomForestRegressor,NaN,standardized values,NaN,NaN,NaN,7,NaN,279,13,5141d92aaf154522bcf28dcc3f492a70,standardized values,from sklearn.ensemble import RandomForestRegre...,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
8,11,mlflow_demo2,0.315244,MLPRegressor,NaN,standardized values,"(10, 10)",NaN,NaN,NaN,1343,NaN,13,0cb931332212452b952920a18c4deee7,standardized values,from sklearn.neural_network import MLPRegressor,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
9,11,mlflow_demo2,0.500909,XGBRegressor,NaN,standardized values,NaN,NaN,0.058512491882074746,6,NaN,453,13,9b3a4e1267c3491fa13368c0dfe26041,standardized values,from xgboost import XGBRegressor,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL


In [14]:
df[['params_alpha','params_l1_ratio','params_learning_rate','params_max_depth']] = \
    df[['params_alpha','params_l1_ratio','params_learning_rate','params_max_depth']].apply(pd.to_numeric)

In [15]:
df.dtypes

experiment_id                 object
experiment_name               object
metrics_r2                   float64
params_algorithm              object
params_alpha                 float64
params_data_set_type          object
params_hidden_layer_sizes     object
params_l1_ratio              float64
params_learning_rate         float64
params_max_depth             float64
params_max_iter               object
params_n_estimators           object
params_random_state           object
run_uuid                      object
tags_data_set_type            object
tags_estimator                object
tags_mlflow.source.name       object
tags_mlflow.source.type       object
dtype: object

## Display experiment results sorted by metric

In [16]:
df.sort_values('metrics_r2',ascending=False)

,experiment_id,experiment_name,metrics_r2,params_algorithm,params_alpha,params_data_set_type,params_hidden_layer_sizes,params_l1_ratio,params_learning_rate,params_max_depth,params_max_iter,params_n_estimators,params_random_state,run_uuid,tags_data_set_type,tags_estimator,tags_mlflow.source.name,tags_mlflow.source.type
9,11,mlflow_demo2,0.500909,XGBRegressor,NaN,standardized values,NaN,NaN,0.058512,6.0,NaN,453,13,9b3a4e1267c3491fa13368c0dfe26041,standardized values,from xgboost import XGBRegressor,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
4,11,mlflow_demo2,0.500365,XGBRegressor,NaN,raw values,NaN,NaN,0.058512,6.0,NaN,453,13,a0cf7b8964da457784940f13d09dae79,raw values,from xgboost import XGBRegressor,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
14,11,mlflow_demo2,0.499639,XGBRegressor,NaN,Min/Max values,NaN,NaN,0.058512,6.0,NaN,453,13,d1ce57871a71479b8a637890a9c5735b,Min/Max values,from xgboost import XGBRegressor,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
7,11,mlflow_demo2,0.377560,RandomForestRegressor,NaN,standardized values,NaN,NaN,NaN,7.0,NaN,279,13,5141d92aaf154522bcf28dcc3f492a70,standardized values,from sklearn.ensemble import RandomForestRegre...,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
12,11,mlflow_demo2,0.377378,RandomForestRegressor,NaN,Min/Max values,NaN,NaN,NaN,7.0,NaN,279,13,208249b08a3b4186998901002b4784aa,Min/Max values,from sklearn.ensemble import RandomForestRegre...,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
2,11,mlflow_demo2,0.377038,RandomForestRegressor,NaN,raw values,NaN,NaN,NaN,7.0,NaN,279,13,ee75aa84c04941769576936c709cebb2,raw values,from sklearn.ensemble import RandomForestRegre...,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
5,11,mlflow_demo2,0.341047,ExtraTreesRegressor,NaN,raw values,NaN,NaN,NaN,7.0,NaN,279,13,febda8ffd05f4ec8a3ef4e2ae15acf61,raw values,from sklearn.ensemble import ExtraTreesRegressor,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
10,11,mlflow_demo2,0.341047,ExtraTreesRegressor,NaN,standardized values,NaN,NaN,NaN,7.0,NaN,279,13,e2fc2eb0fc6746d79b55b53e137defde,standardized values,from sklearn.ensemble import ExtraTreesRegressor,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
15,11,mlflow_demo2,0.341047,ExtraTreesRegressor,NaN,Min/Max values,NaN,NaN,NaN,7.0,NaN,279,13,9f4d134c89204c30ae2940a2b9be15b6,Min/Max values,from sklearn.ensemble import ExtraTreesRegressor,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
8,11,mlflow_demo2,0.315244,MLPRegressor,NaN,standardized values,"(10, 10)",NaN,NaN,NaN,1343,NaN,13,0cb931332212452b952920a18c4deee7,standardized values,from sklearn.neural_network import MLPRegressor,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL


In [17]:
df.shape

(16, 18)